In [1]:
import pandas as pd
from statsmodels.tsa.ardl import ardl_select_order
import numpy as np

import sys
sys.path.append("../src/model")

from ARDL_model import ARDL_model_func

In [2]:
DATA_PATH = "../data/CLEAN"

df_box_offices = pd.read_csv(DATA_PATH+"/Violent_Movies_final.tsv", sep="\t")

In [3]:
# Testing out the ARDL model: 
weekly_revenue_films = ARDL_model_func(df_box_offices)

In [4]:
# QUESTIONS
# 1) Only take the years 1950-2012 ? In other years we have nearly no weekly values ? Due to wars ? 
# 2) Fill all missing weeks with 0 films and 0 box_office_revenue


weekly_revenue_films.head()

,Year,Week,no. films released,Box office revenue
0,1913,33,1,980000.0
1,1914,46,1,87028.0
2,1915,6,1,50000000.0
3,1923,47,1,4168790.0
4,1924,49,1,274827.0


In [5]:
EXOG = weekly_revenue_films
EXOG

,Year,Week,no. films released,Box office revenue
0,1913,33,1,980000.0
1,1914,46,1,87028.0
2,1915,6,1,50000000.0
3,1923,47,1,4168790.0
4,1924,49,1,274827.0
...,...,...,...,...
1824,2012,36,5,312845733.0
1825,2012,38,1,10473039.0
1826,2012,39,1,136513833.0
1827,2012,41,1,2005099.0


In [13]:
# Setting the time frame for the auto-regressive part
max_auto_lag = 4            # take into account max. 4 previous timesteps

# Setting the time span for the distributed lag part
max_film_lag = 4            # take into account max. 4 previous timesteps
max_unemployment_lag = 1    # take into account max 1 previous timestep

# Include time-fixed effects
#time_fixed = True

# Include additional confounding factors
#include_confounding = True

# Get indicator variables for the year-week
EXOG["Year-Week"] = EXOG["Year"].astype(str) + "-" + EXOG["Week"].astype(str)
EXOG

,Year,Week,no. films released,Box office revenue,Year-Week
0,1913,33,1,980000.0,1913-33
1,1914,46,1,87028.0,1914-46
2,1915,6,1,50000000.0,1915-6
3,1923,47,1,4168790.0,1923-47
4,1924,49,1,274827.0,1924-49
...,...,...,...,...,...
1824,2012,36,5,312845733.0,2012-36
1825,2012,38,1,10473039.0,2012-38
1826,2012,39,1,136513833.0,2012-39
1827,2012,41,1,2005099.0,2012-41


In [18]:
# Create time dummies for weekly time-fixed effects
time_dummies = pd.get_dummies(EXOG["Year-Week"], drop_first=True).astype(int)
EXOG_with_dummies = pd.concat([EXOG.drop(columns="Year-Week"), time_dummies], axis=1)
EXOG_with_dummies.head()

,Year,Week,no. films released,Box office revenue,1914-46,1915-6,1923-47,1924-49,1925-53,1926-52,...,2012-35,2012-36,2012-38,2012-39,2012-4,2012-41,2012-42,2012-6,2012-7,2012-8
0,1913,33,1,980000.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1914,46,1,87028.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1915,6,1,50000000.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1923,47,1,4168790.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1924,49,1,274827.0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
random_integers = np.random.randint(0, 100, size=1829)  # Adjust the range (0, 100) as needed

# Create the DataFrame
violence = pd.DataFrame({'violence_score': random_integers})

In [20]:
print(EXOG_with_dummies.dtypes)

Year                    int64
Week                    int64
no. films released      int64
Box office revenue    float64
1914-46                 int64
                       ...   
2012-41                 int64
2012-42                 int64
2012-6                  int64
2012-7                  int64
2012-8                  int64
Length: 1832, dtype: object


In [21]:
# Automatically select lag order based on AIC
# Call ardl_select_order function
selected_order = ardl_select_order(
    endog=violence['violence_score'], 
    exog=EXOG_with_dummies, 
    maxlag=max_auto_lag, 
    maxorder={"Box office revenue": max_film_lag}, 
    ic='aic'
)

/Users/raphael_hellmann/anaconda3/envs/alligators/lib/python3.12/site-packages/statsmodels/tsa/ardl/model.py:455: SpecificationWarning: exog contains variables that are missing from the order dictionary.  Missing keys: 2005-36, 1992-19, 1995-17, 2006-20, 1992-39, 1986-26, 1997-1, 2011-49, 2008-20, 2007-7, 2007-17, 2002-1, 1981-21, 2007-8, 2008-33, 2010-9, 2006-12, 1987-40, 1993-16, 2010-16, 1985-1, 1987-32, 1966-8, 1995-7, 2009-45, 2009-8, 1992-45, 1985-40, 2009-6, 1956-30, 2008-42, 2011-14, 2010-23, 2010-14, 1983-49, 2003-2, 2004-29, 1963-25, 1985-18, 2012-26, 1989-32, 1948-35, 2010-25, 1994-40, 2000-14, 1989-41, 2004-3, 1986-13, 1942-52, 2009-43, 1986-2, 2004-46, 2007-50, 1983-10, 2011-51, 1951-8, 1979-43, 1984-35, 1987-21, 2011-39, 1955-35, 1992-43, 2000-5, 2003-8, 1990-18, 1945-22, 1976-41, 1989-12, 2007-36, 1962-40, 1988-27, 1980-20, 2004-26, 1984-49, 2009-20, 1981-30, 2007-38, 2007-49, 2004-44, 1980-47, 1997-33, 1986-9, 1971-6, 2002-39, 1987-11, 1988-9, 1956-4, 1977-50, 1943-1, 1

In [28]:
print(selected_order.ar_lags)

[1, 2]


In [5]:
box_revenues_clean = df_box_offices.dropna()

In [9]:
weekly_revenues = box_revenues_clean.groupby(["Year", "Week"])["Box office revenue"].sum().reset_index()
weekly_no_films = box_revenues_clean.groupby(["Year", "Week"]).size().reset_index(name="no. films released")

weekly_films_revenues = pd.merge(weekly_no_films, weekly_revenues, on=["Year", "Week"], how="left") 
weekly_films_revenues_sorted = weekly_films_revenues.sort_values(["Year", "Week"], ascending=True)

In [10]:
weekly_films_revenues_sorted.head(50)

,Year,Week,no. films released,Box office revenue
0,1913,33,1,980000.0
1,1914,46,1,87028.0
2,1915,6,1,50000000.0
3,1923,47,1,4168790.0
4,1924,49,1,274827.0
5,1925,53,1,10738000.0
6,1926,52,1,1000000.0
7,1930,46,1,8000000.0
8,1930,52,1,780000.0
9,1931,47,1,12000000.0
